<a href="https://colab.research.google.com/github/alexandrebatista/mips-spark-optimization/blob/master/trabfinal_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Como baixar dados do Kaggle direto no Colab:

https://www.kaggle.com/general/74235

Pré-processando os dados:

https://www.kaggle.com/morrisb/how-to-recommend-anything-deep-recommender


In [1]:
! pip install -q kaggle

from google.colab import files

files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
#! kaggle datasets list


Saving kaggle.json to kaggle.json


In [2]:
!kaggle datasets download -d netflix-inc/netflix-prize-data

 98% 671M/683M [00:06<00:00, 98.8MB/s]
100% 683M/683M [00:06<00:00, 107MB/s] 


In [6]:
!mkdir train
!unzip netflix-prize-data.zip -d train

mkdir: cannot create directory ‘train’: File exists
Archive:  netflix-prize-data.zip
replace train/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: train/README            
  inflating: train/combined_data_1.txt  
  inflating: train/combined_data_2.txt  
  inflating: train/combined_data_3.txt  
  inflating: train/combined_data_4.txt  
  inflating: train/movie_titles.csv  
  inflating: train/probe.txt         
  inflating: train/qualifying.txt    


In [7]:
from collections import deque
import pandas as pd

In [8]:
# Load single data-file
df_raw = pd.read_csv('train/combined_data_1.txt', header=None, names=['User', 'Rating', 'Date'], usecols=[0, 1, 2])

# Find empty rows to slice dataframe for each movie
tmp_movies = df_raw[df_raw['Rating'].isna()]['User'].reset_index()
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

# Shift the movie_indices by one to get start and endpoints of all movies
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)

# Gather all dataframes
user_data = []

# Iterate over all movies
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
  
  # Check if it is the last movie in the file
  if df_id_1<df_id_2:
    tmp_df = df_raw.loc[df_id_1+1:df_id_2-1].copy()
  else:
    tmp_df = df_raw.loc[df_id_1+1:].copy()
      
  # Create movie_id column
  tmp_df['Movie'] = movie_id
  
  # Append dataframe to list
  user_data.append(tmp_df)

# Combine all dataframes
df = pd.concat(user_data)
del user_data, df_raw, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
print('Shape User-Ratings:\t{}'.format(df.shape))
df.sample(5)

Shape User-Ratings:	(24053764, 4)


,User,Rating,Date,Movie
15915737,179167,5.0,2005-01-10,3102
20905106,1884092,5.0,2003-11-23,3932
10410972,1747464,4.0,2003-05-14,2012
3099623,793748,2.0,2003-12-25,571
3160635,1442467,5.0,2004-06-24,571
